# Using Supabase as a vector store with LlamaIndex in Python

## Before you get started

You will need to have a supabase database with the vector store extension enabled and can follow the llamaindex [docs](https://docs.llamaindex.ai/en/stable/examples/vector_stores/SupabaseVectorIndexDemo/) to get there. Or you can just follow me.

![](assets/pgvector.jpg)

> Important: For your supabase project's password be sure to use a password that has no special characters or it will break the connection string you'll be using later.

# Prerequisites

## Install the packages

Note that I used Python 3.10.13 and this all worked for me. I tried it in 3.11 and 3.12 and didn't have the same luck.

In [ ]:
!pip install llama-index
!pip install llama-index-vector-stores-supabase
!pip install python-dotenv
!pip install llama-index-llms-openai

### Get your `.env` file ready

We will use a `.env` to store two pieces of information. Note that it doesn't exist because it's in `.gitignore` so that you don't accidentally commit your secrets. The file should read with two lines like this:

```
OPENAI_API_KEY=sk-...
POSTGRES_CONNECTION_STRING=postgresql:...
```

The POSTGRES_CONNECTION_STRING is the connection string to your supabase database which has the format `postgresql://<user>:<password>@<host>:<port>/<db_name>` and you can get it by going first to your project settings and then to Database and then to Connection String.

![](assets/projsettings.jpg)

![](assets/postgres.jpg)


### Load the basic packages and key information


In [2]:
import logging
import sys
import os

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import SimpleDirectoryReader, Document, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.supabase import SupabaseVectorStore
from dotenv import load_dotenv
import textwrap

load_dotenv()  # This loads the variables from .env
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
connection_string = os.getenv("POSTGRES_CONNECTION_STRING")

# Writing to the Supabase vector store

### Grab the documents in a directory and prep them for llama index

In [ ]:
collection_name = "maeda_demo"
documents = SimpleDirectoryReader("./data/john_maeda/").load_data()
print(f"> {len(documents)} doc chunks written to vecs schema{collection_name} in supabase")

### Write documents to `collection_name` in supabase

> Warning: Be sure to delete the `collection_name` before adding documents to it, or else you'll keep accruing documents into that collection. You can either delete it manually like below, or use the code at the end of this notebook.

![](assets/deletecollection.jpg)

Let's go ahead and write the chunks to supabase.

In [4]:
vector_store = SupabaseVectorStore(
    postgres_connection_string=(
       connection_string
    ),
    collection_name=collection_name,
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### You're done! Now you can read from the database any time you want.

# Reading from the Supabase vector store

### Let's plug llamaindex into the Supabase vector store

In [5]:
from llama_index.core import StorageContext, load_index_from_storage

vector_store = SupabaseVectorStore(
    postgres_connection_string=(
       connection_string
    ),
    collection_name=collection_name,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store = vector_store, storage_context=storage_context)

## Query the index however you like

In [ ]:
import warnings

# I added this to suppress a warning that stressed me out
warnings.filterwarnings("ignore", message="Query does not have a covering index for cosine_distance")

query_engine = index.as_query_engine()
query = """What is Maeda's george clooney inspiration? 
  What's the specific takeaway? 
  Don't make up any detail that isn't in the text.
"""
response = query_engine.query(query)
print(f"> {query}")
print(textwrap.fill(str(response), 100))

It's a lot of fun to talk to oneself ... sorta.

In [ ]:
query = """What did the author do growing up? 
  Say 'I don't know' if you don't know.
"""
response = query_engine.query(query)
print(f"> {query}")
print(textwrap.fill(str(response), 100))

## Retrieve the actual chunks related to the query

This is some utility code for setup.

In [13]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import QueryBundle
from IPython.display import Markdown
def get_retrieved_nodes(query_str, vector_top_k=5, reranker_top_n=2, with_reranker=False):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(
            choice_batch_size=3,
            top_n=reranker_top_n,
        )
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)

    return retrieved_nodes

def pretty_print(results):
    markdown_str = "| Score | Text |\n|-------|------|\n"
    for result in results:
        # Replace line breaks with space for Markdown format
        text = result["Text"].replace("\n", " ")
        markdown_str += "| {} | {} |\n".format(result["Score"], text)
    display(Markdown(markdown_str))

def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(result_dicts)

### Without Re-ranking it's not working that hard

In [ ]:
my_new_query = "What did John Maeda think about the role of technology in art?"
nodes = get_retrieved_nodes(my_new_query)
visualize_retrieved_nodes(nodes)

### With Re-ranking it is a bit more accurate

In [ ]:
new_nodes = get_retrieved_nodes(
    my_new_query,
    vector_top_k=3,
    with_reranker=True,
)
visualize_retrieved_nodes(new_nodes)

### Using a "tree summarizer" approach in conjunction with re-ranking

In [ ]:
from llama_index.core.postprocessor import LLMRerank
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=2,
        )
    ],
    verbose=True,
    response_mode="tree_summarize",
)
response = query_engine.query(
    my_new_query,
)

print(textwrap.fill(str(response), 80))


### Using citations to get specific details

In [ ]:
from llama_index.core.query_engine import CitationQueryEngine

query_text = "I would like to hear your thoughts about art and technology. "

response_dict = {}  # Initialize an empty dictionary for response

query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    citation_chunk_size = 1024
)
myq = query_text # + " If the facts are not available say, 'I do not know.'\n"
response = query_engine.query(myq)
print(textwrap.fill(f"Q: {myq}",80))
print("")
print(textwrap.fill(f"ai-johnmaeda response: {response}",80))
print(f"-- with {len(response.source_nodes)} citations.")
print("---")
# response_text = f"query: {myq} response: {response}\n"
# # print out all nodes text with the enumeration of node
# for i, node in enumerate(response.source_nodes):
#     response_text += f"**{i}**: {node.node.get_text()}\n"

# Create a dictionary to represent the response
response_dict = {
    "text": f"Query: {myq}",
    "citations": []
}

# Extract information from each node and add to the response dictionary
for i, node in enumerate(response.source_nodes):
    node_text = node.node.get_text()
    # extract metadata
    metadata = node.node.metadata
    print(f"---\n[{i}]\n")
    for key, value in metadata.items():
        print(f"{key}: {value}")
    print("")
    # Assuming node_text is a dictionary or can be converted to one
    # You may need to adjust this based on the actual structure of node.node.get_text()
    print(node_text)
    response_dict["citations"].append(node_text)


# Congratulations! You've made it through the notebook!

---

## Bonus: Supabase vec schema utilities that might help you

### List all tables in the vecs schema

In [ ]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(connection_string)

try:
    with conn.cursor() as cur:
        cur.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'vecs'
        """)
        tables = cur.fetchall()
        
        print("Tables in 'vecs' schema:")
        for table in tables:
            print(f"- {table[0]}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

### Delete a table in the vecs schema

In [ ]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(connection_string)
conn.autocommit = True

try:
    with conn.cursor() as cur:
        drop_table_sql = f'DROP TABLE IF EXISTS vecs."{collection_name}" CASCADE'
        cur.execute(drop_table_sql)
        print(f"Table 'vecs.{table_to_delete}' has been deleted successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()